In [1]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer

print(f"PyTorch Version: {torch.__version__}")

# 设置一个随机种子，以保证实验结果的可复现性
torch.manual_seed(42)

PyTorch Version: 2.8.0+cpu


# 独热编码部分

我们先创建一个微型词典。
一般来说，我们定义的词库就是这样。
一个词对应一个数字

In [2]:
vocab = {"我": 0, "爱": 1, "苹果": 2, "香蕉": 3}
vocab_size = len(vocab)

In [3]:
def to_one_hot(word,vocab):
    vec = torch.zeros(len(vocab))
    num = vocab[word]
    vec[num] = 1
    return vec


In [4]:
word_a = "苹果"
word_b = "香蕉"
one_hot_a = to_one_hot(word_a, vocab)
one_hot_b = to_one_hot(word_b, vocab)

print(f"'{word_a}' 的独热编码是: {one_hot_a}")
print(f"'{word_b}' 的独热编码是: {one_hot_b}")
print("-" * 20)

'苹果' 的独热编码是: tensor([0., 0., 1., 0.])
'香蕉' 的独热编码是: tensor([0., 0., 0., 1.])
--------------------


F.cosine_similarity(vec1, vec2) 期望的 vec1 和 vec2 都是一个向量列表，形状为 [N, D]，其中 N 是批次大小（列表里有N个向量），D 是每个向量的维度。
我们就使用unsqueeze方法，假设批次大小为1

In [5]:
similarity = F.cosine_similarity(one_hot_a.unsqueeze(0), one_hot_b.unsqueeze(0))


print(f"'{word_a}' 和 '{word_b}' 的余弦相似度是: {similarity.item():.2f}")
print("结论：正如预期，它们之间毫无关联。")

'苹果' 和 '香蕉' 的余弦相似度是: 0.00
结论：正如预期，它们之间毫无关联。


In [6]:
#cosine_similarity的本质：
dot_product = torch.dot(one_hot_a, one_hot_b)
norm_a = torch.linalg.norm(one_hot_a)
norm_b = torch.linalg.norm(one_hot_b)
similarity_manual = dot_product / (norm_a * norm_b)
print(f"手动实现的相似度: {similarity_manual.item():.2f}")


手动实现的相似度: 0.00


# 感受稠密向量

In [7]:
import gensim
from huggingface_hub import hf_hub_download

print("正在从Hugging Face Hub下载社区验证的腾讯词向量轻量版...")
try:
    # 这是由shibing624上传、社区广泛使用的腾讯词向量轻量版
    # 我们已经亲自验证过其repo_id和filename的有效性
    model_path = hf_hub_download(
        repo_id="shibing624/text2vec-word2vec-tencent-chinese", 
        filename="light_Tencent_AILab_ChineseEmbedding.bin"
    )
    
    # 使用Gensim加载这个二进制(.bin)格式的模型
    gensim_model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)
    
    print(f"\n模型加载成功！")
    print(f"词典规模: {len(gensim_model.index_to_key)} 个词")
    print(f"向量维度: {gensim_model.vector_size}")

except Exception as e:
    print(f"\n加载模型失败。请检查网络连接或相关库。错误: {e}")
    gensim_model = None

正在从Hugging Face Hub下载社区验证的腾讯词向量轻量版...

模型加载成功！
词典规模: 143613 个词
向量维度: 200


In [8]:
if gensim_model:
    # 1. 直接从模型中获取 "苹果", "香蕉", "手机" 的词向量
    # Gensim模型可以直接像字典一样查询
    vec_apple = torch.from_numpy(gensim_model["苹果"])
    vec_banana = torch.from_numpy(gensim_model["香蕉"])
    vec_phone = torch.from_numpy(gensim_model["手机"])

    # 2. 计算相似度
    sim_apple_banana = F.cosine_similarity(vec_apple.unsqueeze(0), vec_banana.unsqueeze(0))
    sim_apple_phone = F.cosine_similarity(vec_apple.unsqueeze(0), vec_phone.unsqueeze(0))
    sim_banana_phone = F.cosine_similarity(vec_banana.unsqueeze(0), vec_phone.unsqueeze(0))

    print(f"\n'苹果' vs '香蕉' (水果类比) 相似度: {sim_apple_banana.item():.4f}")
    print(f"'苹果' vs '手机' (品牌歧义) 相似度: {sim_apple_phone.item():.4f}")
    print(f"'香蕉' vs '手机' (无关对比) 相似度: {sim_banana_phone.item():.4f}")

    # 3. 验证经典案例: 国王 - 男人 + 女人 ≈ 女王
    result_vec = gensim_model.most_similar(positive=['国王', '女人'], negative=['男人'], topn=3)
    print(f"\n计算 '国王' - '男人' + '女人'，最相似的词是: {result_vec}")
    
    print("\n结论：基于词的嵌入模型，成功地、清晰地捕捉到了丰富的语义关系！")



'苹果' vs '香蕉' (水果类比) 相似度: 0.6102
'苹果' vs '手机' (品牌歧义) 相似度: 0.5659
'香蕉' vs '手机' (无关对比) 相似度: 0.3413

计算 '国王' - '男人' + '女人'，最相似的词是: [('王后', 0.7049503326416016), ('爵士', 0.6559656262397766), ('王子', 0.6505958437919617)]

结论：基于词的嵌入模型，成功地、清晰地捕捉到了丰富的语义关系！


C:\Users\kkkkkk\AppData\Local\Temp\ipykernel_34112\1593690901.py:4: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:209.)
  vec_apple = torch.from_numpy(gensim_model["苹果"])


In [13]:
# 这里注意一下stack的用法
sentence1 = ["我", "打", "你"]
vecs1 = [torch.from_numpy(gensim_model[w]) for w in sentence1 if w in gensim_model]
tensor = torch.stack(vecs1)  # 把list里的多个张量拼成一个多维张量
print(tensor.shape)

torch.Size([3, 200])


In [18]:
#4. 实验三：静态词向量的“阿喀琉斯之踵” —— 语序的丢失 (最终版)
# ===================================================================
if gensim_model:
    sentence1 = ["我", "打", "你"]
    sentence2 = ["你", "打", "我"]

    # --- TODO: 工匠，这是本 Notebook 最具戏剧性的一幕 ---
    # 2. 将句子中的词转换为词向量列表
    vecs1 = [torch.from_numpy(gensim_model[w]) for w in sentence1 if w in gensim_model]
    vecs2 = [torch.from_numpy(gensim_model[w]) for w in sentence2 if w in gensim_model]

    if len(vecs1) == 3 and len(vecs2) == 3:
        # 3. 求平均得到句向量
        sent_vec1 = torch.stack(vecs1).mean(dim=0)
        sent_vec2 = torch.stack(vecs2).mean(dim=0)

        # 4. 比较两个句向量是否相等
        are_they_equal = torch.allclose(sent_vec1, sent_vec2)
        
        print(f"\n句子 '{''.join(sentence1)}' 的向量表示 (前10维): \n{sent_vec1[:10]}")
        print(f"句子 '{''.join(sentence2)}' 的向量表示 (前10维): \n{sent_vec2[:10]}")
        print("-" * 20)
        print(f"这两个句向量是否完全相等? -> {are_they_equal}")
        print("俩个句子显示相同，语序信息被完全丢失了。")


句子 '我打你' 的向量表示 (前10维): 
tensor([ 0.3039, -0.2574, -0.0179,  0.0484,  0.0692, -0.1234,  0.0205,  0.0801,
         0.1206,  0.1045])
句子 '你打我' 的向量表示 (前10维): 
tensor([ 0.3039, -0.2574, -0.0179,  0.0484,  0.0692, -0.1234,  0.0205,  0.0801,
         0.1206,  0.1045])
--------------------
这两个句向量是否完全相等? -> True
俩个句子显示相同，语序信息被完全丢失了。


# RNN部分


In [17]:
if gensim_model and 'vecs1' in locals():
    # 1. 准备RNN的输入
    # RNN需要 [sequence_length, batch_size, input_size] 格式的输入
    # 这里我们 batch_size=1
    sentence1 = ["我", "打", "你"]
    sentence2 = ["你", "打", "我"]

    vecs1 = [torch.from_numpy(gensim_model[w]) for w in sentence1 if w in gensim_model ]
    vecs2 = [torch.from_numpy(gensim_model[w]) for w in sentence2 if w in gensim_model ]
    rnn_input1 = torch.stack(vecs1).unsqueeze(1) # 形状变为 [3, 1, embedding_dim]
    rnn_input2 = torch.stack(vecs2).unsqueeze(1) # 形状变为 [3, 1, embedding_dim]
    
    embedding_dim = rnn_input1.shape[2]
    hidden_size = 64 # 我们可以任意定义隐藏层大小

    
    rnn = nn.RNN(input_size=embedding_dim, hidden_size=hidden_size)

    
    # RNN会返回所有时间步的输出(outputs)和最后一个时间步的隐藏状态(hidden_state)
    outputs1, hidden_state1 = rnn(rnn_input1)
    outputs2, hidden_state2 = rnn(rnn_input2)

    # 我们关心的是最后一个隐藏状态，它代表了对整个句子的编码
    final_repr1 = hidden_state1
    final_repr2 = hidden_state2
    
    # 4. 比较两个最终的隐藏状态是否不同
    are_they_different = not torch.allclose(final_repr1, final_repr2)

    print(f"RNN对 '{''.join(sentence1)}' 的最终编码 (前10维): \n{final_repr1.squeeze()[:10]}")
    print(f"RNN对 '{''.join(sentence2)}' 的最终编码 (前10维): \n{final_repr2.squeeze()[:10]}")
    print("-" * 20)
    print(f"这两个RNN编码是否不同? -> {are_they_different}")

RNN对 '我打你' 的最终编码 (前10维): 
tensor([ 0.0545, -0.1765,  0.0484, -0.2835, -0.1166, -0.0811, -0.1037, -0.1684,
         0.0372, -0.6473], grad_fn=<SliceBackward0>)
RNN对 '你打我' 的最终编码 (前10维): 
tensor([ 0.1967, -0.1812,  0.0270, -0.2130, -0.0457,  0.0019, -0.0922, -0.2479,
        -0.0234, -0.6513], grad_fn=<SliceBackward0>)
--------------------
这两个RNN编码是否不同? -> True
结论：奇迹发生！RNN成功地区分了不同的语序。


⚠️ **一个至关重要的澄清：这真的证明了什么吗？** ⚠️

你可能会有一个疑问：这个RNN未经训练，权重都是随机的，不同的输入顺序，经过不同的随机计算，得到不同的结果不是很正常吗？

我们必须明确：
1. 这个实验**没有**证明RNN'理解'了语序。因为未经训练，它的输出是无意义的。
2. 这个实验**证明**的是：RNN的**数学结构**（递归）是'**顺序敏感**'的。它从根本上具备了区分不同顺序的能力。
3. 与之形成鲜明对比的是之前展示的word2vec，'词袋模型'（求平均），它的数学结构是'顺序不敏感'的，它在物理上就**不可能**区分语序。
----
我们展示的不是一个'训练好的结果'，而是一个'具备潜力的结构'。训练的作用，就是将这种'区分能力'，从'随机地区分'，升华为'有意义地区分'。

## 打开RNN黑箱
$ h_t = tanh(W_hh * h_{t-1} + W_xh * x_t + b_h) $

我们需要初始化俩个矩阵，
对当前时间步的输入（在我们的情景下，也就是当前处理的词。），进行处理的矩阵。
以及对上一步隐藏层，进行处理的矩阵。
forward根据公式写即可，所以整体来说还是蛮简单的。

----
接着我们进行验证。
我们用到上一个小节，“我打你”和“你打我”俩个句子的RNN_input

进入模型，返回结果，进行对比。
我们再次注意模型中的数据流动，for循环下一共有三次处理，均是对hidden_t的处理。

**让我们以`rnn_input1`（代表“我打你”）为例，进行一次“调试”：**

`rnn_input1` 的形状是 `[3, 1, 200]`。`for word_vec in rnn_input1:` 这个循环会执行**3次**。

1.  **循环开始前：**
    *   `hidden_t = initial_hidden`
    *   `hidden_t` 是我们的“当前记忆”，现在是全零的“空白记忆”。形状：`[1, 64]`。

2.  **第一次循环 (处理“我”)：**
    *   `word_vec` 是“我”的词向量。形状：`[1, 200]`。
    *   执行 `hidden_t = my_rnn(word_vec, hidden_t)`。
    *   **内部发生了什么？** `my_rnn`接收了“我”的向量和“空白记忆”，通过 `tanh(W*x + W*h + b)` 的计算，产出了一个新的`hidden_t`。
    *   **结果：** `hidden_t` 现在是 `h_我`，它包含了“我”的信息。形状：`[1, 64]`。

3.  **第二次循环 (处理“打”)：**
    *   `word_vec` 是“打”的词向量。形状：`[1, 200]`。
    *   执行 `hidden_t = my_rnn(word_vec, hidden_t)`。
    *   **内部发生了什么？** `my_rnn`接收了“打”的向量和**刚刚更新的** `h_我`，通过同样的公式，产出了一个新的`hidden_t`。
    *   **结果：** `hidden_t` 现在是 `h_我打`，它融合了“我”和“打”的信息。形状：`[1, 64]`。

4.  **第三次循环 (处理“你”)：**
    *   `word_vec` 是“你”的词向量。形状：`[1, 200]`。
    *   执行 `hidden_t = my_rnn(word_vec, hidden_t)`。
    *   **内部发生了什么？** `my_rnn`接收了“你”的向量和**刚刚更新的** `h_我打`，产出了最终的`hidden_t`。
    *   **结果：** `hidden_t` 现在是 `h_我打你`，它代表了对整个句子的理解。形状：`[1, 64]`。

5.  **循环结束后：**
    *   执行 `my_final_repr1 = hidden_t`。
    *   我们将这个最终的、包含了整个句子信息的“记忆”，赋值给`my_final_repr1`。

对`rnn_input2`的处理过程完全一样，只是输入的顺序是“你”、“打”、“我”，因此最终得到的`my_final_repr2`必然不同。

In [24]:
rnn_input1.shape

torch.Size([3, 1, 200])

In [25]:

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        
        # 权重矩阵
        self.W_xh = nn.Linear(input_size, hidden_size, bias=False)
        self.W_hh = nn.Linear(hidden_size, hidden_size) # 包含偏置
        
        # 激活函数
        self.activation = nn.Tanh()

    def forward(self, input_tensor, hidden_tensor):
        """
        这个 forward 方法只处理一个时间步的计算。
        """
        combined = self.W_xh(input_tensor) + self.W_hh(hidden_tensor)
        hidden_tensor = self.activation(combined)
        return hidden_tensor

# 手动循环，验证我们自己的RNN 
my_rnn = SimpleRNN(embedding_dim, hidden_size)
initial_hidden = torch.zeros(1, hidden_size) 


hidden_t = initial_hidden
for word_vec in rnn_input1:
    hidden_t = my_rnn(word_vec, hidden_t)
my_final_repr1 = hidden_t

# 处理句子2
hidden_t = initial_hidden
for word_vec in rnn_input2:
    hidden_t = my_rnn(word_vec, hidden_t)
my_final_repr2 = hidden_t


# 比较结果
are_they_different_my_rnn = not torch.allclose(my_final_repr1, my_final_repr2)

print(f"我们自己实现的RNN，两个句子的编码是否不同? -> {are_they_different_my_rnn}")


我们自己实现的RNN，两个句子的编码是否不同? -> True
